# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1: Marta Castillo Galán
*   Alumno 2: Unai Marín Etxebarria
*   Alumno 3: Laura Molinos Mayo
*   Alumno 4: Raúl Murillo Gallego






---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

In [72]:
#if IN_COLAB:
  #%pip install gym==0.17.3
  #%pip install git+https://github.com/Kojoley/atari-py.git
  #%pip install keras-rl2==1.0.5
  #%pip install tensorflow==2.8
#else:
  #%pip install gym==0.17.3
  #%pip install git+https://github.com/Kojoley/atari-py.git
  #%pip install pyglet==1.5.0
  #%pip install h5py==3.1.0
  #%pip install Pillow==9.5.0
  #%pip install keras-rl2==1.0.5
  #%pip install Keras==2.2.4
  #%pip install tensorflow==2.5.3
  ##%pip install torch==2.0.1
  #%pip install agents==1.4.0

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [73]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [74]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [75]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)


1. Implementación de la red neuronal

En otra ejecución hemos utilizado el siguiente modelo:

In [76]:
def build_model(window_length, input_shape, nb_actions):
    model = Sequential()
    model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))
    model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    return model

Con el objetivo de incorporar técnicas de regulación, introducimos dropout después de la primera capa densa. Como ya sabemos, esto podría ayudar a generalizar y evitar el sobreajuste. Además, vamos a ampliar la tercera capa convolucional de 64 a 512 neuronas, a sabiendas de que el número de parámetros incrementará considerablemente.

In [77]:

from tensorflow.keras.layers import  Dropout

def build_model(window_length, input_shape, nb_actions):
    model = Sequential()
    model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))  # (C, H, W) → (H, W, C)

    # Convolucional 1
    model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='same'))
    model.add(Activation('relu'))
    
    # Convolucional 2
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(Activation('relu'))
    
    # Convolucional 3 (ligero cambio en tamaño del kernel)
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))

    model.add(Flatten())

    # Capa densa + regularización
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))  # 25% dropout

    # Salida
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    
    return model


2. Implementación de la solución DQN

Por lo demás, seguimos con la misma estrategia.

In [78]:
# Preparamos memoria y política
memory = SequentialMemory(limit=100000, window_length=WINDOW_LENGTH)
#policy = EpsGreedyQPolicy()  # Exploración simple
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                              value_max=1.0, value_min=0.1,
                              value_test=0.05, nb_steps=100000)

# Creamos el modelo
model = build_model(WINDOW_LENGTH, INPUT_SHAPE, nb_actions)
print(model.summary())

# Creamos el procesador
processor = AtariProcessor()

#Creamos el agente
dqn = DQNAgent(model=model,
               nb_actions=nb_actions,
               policy=policy,
               memory=memory,
               processor=processor,
               nb_steps_warmup=10000,
               gamma=0.99,
               target_model_update=10000,
               train_interval=4,
               delta_clip=1.0)

dqn.compile(Adam(learning_rate=1e-4), metrics=['mae'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_5 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 21, 21, 32)        8224      
_________________________________________________________________
activation_25 (Activation)   (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 11, 11, 64)        32832     
_________________________________________________________________
activation_26 (Activation)   (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 11, 11, 512)       295424    
_________________________________________________________________
activation_27 (Activation)   (None, 11, 11, 512)      

In [79]:
# Entrenamiento inicial
# (0.0.0.2) Pasamos de 5000 a 20.000 pasos
dqn.fit(env, nb_steps=20000, visualize=False, verbose=2)

Training for 20000 steps ...


C:\Users\Unai\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   420/20000: episode: 1, duration: 9.851s, episode steps: 420, steps per second:  43, episode reward:  6.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.471 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1218/20000: episode: 2, duration: 16.181s, episode steps: 798, steps per second:  49, episode reward: 11.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.421 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1844/20000: episode: 3, duration: 12.406s, episode steps: 626, steps per second:  50, episode reward:  8.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.388 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  2676/20000: episode: 4, duration: 17.019s, episode steps: 832, steps per second:  49, episode reward: 10.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.361 [0.000, 5.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  3064/20000: episode: 5, duration: 10.646s, episode steps: 388, steps per second

C:\Users\Unai\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 10162/20000: episode: 16, duration: 26.860s, episode steps: 424, steps per second:  16, episode reward:  8.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 2.342 [0.000, 5.000],  loss: 0.008356, mae: 0.027945, mean_q: 0.058856, mean_eps: 0.909262
 10755/20000: episode: 17, duration: 75.819s, episode steps: 593, steps per second:   8, episode reward:  4.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 2.511 [0.000, 5.000],  loss: 0.006304, mae: 0.020641, mean_q: 0.033484, mean_eps: 0.905878
 11752/20000: episode: 18, duration: 116.499s, episode steps: 997, steps per second:   9, episode reward: 17.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.464 [0.000, 5.000],  loss: 0.007238, mae: 0.022784, mean_q: 0.036257, mean_eps: 0.898732
 12934/20000: episode: 19, duration: 137.926s, episode steps: 1182, steps per second:   9, episode reward: 15.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.548 [0.000, 5.000],  loss: 0.006980, mae: 0.024021, mean_q: 0.0409

In [80]:
#Evaluación en modo test
scores = dqn.test(env, nb_episodes=10, visualize=False)
print("Media de recompensa:", np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 24.000, steps: 1118
Episode 2: reward: 15.000, steps: 963
Episode 3: reward: 10.000, steps: 637
Episode 4: reward: 18.000, steps: 718
Episode 5: reward: 16.000, steps: 1257
Episode 6: reward: 18.000, steps: 784
Episode 7: reward: 18.000, steps: 993
Episode 8: reward: 15.000, steps: 1112
Episode 9: reward: 15.000, steps: 932
Episode 10: reward: 14.000, steps: 973
Media de recompensa: 16.3


In [81]:
#Guardado de pesos
dqn.save_weights('dqn_spaceinvaders_weights_aprende_a_esquivar.h5f', overwrite=True)

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

Es interesante observar la el rendimiento de este algoritmo. Por una parte, es obvio que la media de puntuación obtenida no es suficiente, y por ese lado no hemos conseguido ninguna mejora. Sin embargo, obsérvese que a medida que avanzaba el entrenamiento, el algoritmo a aprendido a sobrevivir durante más tiempo. Uno de los episodios ha llegado a durar 108 segundo, esto es, casi dos minutos. 

De modo que cabe concluir que la red esta aprendiendo a esquivar los ataques enemigos. Esto, en si mismo, no constituye ningun objetivo, ni merece recompensa. Aun así, es una propiedad que todo agente exitoso necesitará, puesto que lo ideal es aprender a no morir, a la par que elimnar enemigos. Por lo tanto, necesitamos una arquitectura que logre captar la habilidad de esquivar balas de este modelo, pero que corrija su incapacidad para matar enemigos.

---